source: https://www.kaggle.com/c/word2vec-nlp-tutorial#part-2-word-vectors

## Introducing Distributed Word Vectors

Now down to the nitty-gritty! First, we read in the data with pandas, as we did in Part 1. Unlike Part 1, we now use unlabeledTrain.tsv, which contains 50,000 additional reviews with no labels. When we built the Bag of Words model in Part 1, extra unlabeled training reviews were not useful. However, since Word2Vec can learn from unlabeled data, these extra 50,000 reviews can now be used.




In [3]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "data/labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

test = pd.read_csv( "data/testData.tsv", header=0, delimiter="\t", quoting=3 )

unlabeled_train = pd.read_csv( "data/unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print ("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



The functions we write to clean the data are also similar to Part 1, although now there are a couple of differences. First, to train Word2Vec it is better not to remove stop words because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors. For this reason, we will make stop word removal optional in the functions below. It also might be better not to remove numbers, but we leave that as an exercise for the reader.



In [4]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review,"lxml").get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

Next, we want a specific input format. Word2Vec expects single sentences, each one as a list of words. In other words, the input format is a list of lists.

It is not at all straightforward how to split a paragraph into sentences. There are all kinds of gotchas in natural language. English sentences can end with "?", "!", """, or ".", among other things, and spacing and capitalization are not reliable guides either. For this reason, we'll use NLTK's punkt tokenizer for sentence splitting. In order to use this, you will need to install NLTK and use nltk.download() to download the relevant training file for punkt.

In [5]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
#nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [6]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

# A minor detail to note is the difference between the "+=" and "append" when it comes to Python lists. In many applications the two are interchangeable, but here they are not. If you are appending a list of lists to another list of lists, "append" will only append the first list; you need to use "+=" in order to join all of the lists at once.

Parsing sentences from training set


/Users/inshique/anaconda/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
/Users/inshique/anaconda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/Users/inshique/anaconda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/inshique/anaconda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/inshique/anaconda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP clie

In [7]:
# Check how many sentences we have in total - should be around 850,000+
print (len(sentences))

795538


In [8]:
print (sentences[0])
print (sentences[1])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


## Training and Saving Your Model

- **Architecture**: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.
- Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
- Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.
- Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.
- Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
- Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.
- Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.


In [9]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2017-03-31 11:56:07,911 : INFO : 'pattern' package not found; tag filters are not available for English
2017-03-31 11:56:07,938 : INFO : collecting all words and their counts
2017-03-31 11:56:07,939 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-03-31 11:56:07,989 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2017-03-31 11:56:08,046 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2017-03-31 11:56:08,097 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types


Training model...


2017-03-31 11:56:08,151 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2017-03-31 11:56:08,210 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2017-03-31 11:56:08,264 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2017-03-31 11:56:08,319 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2017-03-31 11:56:08,377 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2017-03-31 11:56:08,427 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2017-03-31 11:56:08,480 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2017-03-31 11:56:08,531 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2017-03-31 11:56:08,586 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

If you are on a Mac or Linux system, you can use the "top" command from within Terminal (not from within Python) to see if your system is successfully parallelizing while the model is training. Type



In [10]:
#!top -o cpu 

## Exploring the Model Results

Congratulations on making it successfully through everything so far! Let's take a look at the model we created out of our 75,000 training reviews.

The "doesnt_match" function will try to deduce which word in a set is most dissimilar from the others:



In [11]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

Our model is capable of distinguishing differences in meaning! It knows that men, women and children are more similar to each other than they are to kitchens. More exploration shows that the model is sensitive to more subtle differences in meaning, such as differences between countries and cities:



In [12]:
model.doesnt_match("france england germany berlin".split())


'berlin'

In [13]:
model.doesnt_match("paris berlin london austria".split())

'paris'

We can also use the "most_similar" function to get insight into the model's word clusters:



In [14]:
model.most_similar("man")

[('woman', 0.6147068738937378),
 ('lad', 0.6082320213317871),
 ('lady', 0.5901197791099548),
 ('monk', 0.5194621086120605),
 ('chap', 0.5161230564117432),
 ('guy', 0.5109291672706604),
 ('person', 0.5100181698799133),
 ('farmer', 0.5035187005996704),
 ('widow', 0.4974830150604248),
 ('businessman', 0.49612903594970703)]

In [15]:
model.most_similar("queen")

[('princess', 0.660820484161377),
 ('bride', 0.6360553503036499),
 ('maid', 0.6038008332252502),
 ('eva', 0.5977911949157715),
 ('duchess', 0.5974718332290649),
 ('belle', 0.5914064645767212),
 ('victoria', 0.5819592475891113),
 ('angela', 0.5788499116897583),
 ('showgirl', 0.5780096650123596),
 ('maria', 0.5723618865013123)]

In [41]:
model.most_similar("awful")

[('terrible', 0.7751122713088989),
 ('atrocious', 0.7426633834838867),
 ('horrible', 0.7383997440338135),
 ('dreadful', 0.7098507881164551),
 ('abysmal', 0.6990411877632141),
 ('horrendous', 0.6727656126022339),
 ('horrid', 0.6714054346084595),
 ('appalling', 0.671180784702301),
 ('lousy', 0.6432656049728394),
 ('amateurish', 0.6219732165336609)]

In [44]:
model.most_similar("book")

[('novel', 0.7439521551132202),
 ('books', 0.7401041984558105),
 ('novels', 0.5317251682281494),
 ('manga', 0.5278310775756836),
 ('poem', 0.4906112551689148),
 ('relief', 0.4516502022743225),
 ('text', 0.4481984078884125),
 ('description', 0.44743192195892334),
 ('review', 0.4395514726638794),
 ('adaptation', 0.43863749504089355)]

In [17]:
model.syn1neg.shape

(16490, 300)

# Part 3

## Numeric Representations of Words

Now that we have a trained model with some semantic understanding of words, how should we use it? If you look beneath the hood, the Word2Vec model trained in Part 2 consists of a feature vector for each word in the vocabulary, stored in a numpy array called "syn0":

The number of rows in syn0 is the number of words in the model's vocabulary, and the number of columns corresponds to the size of the feature vector, which we set in Part 2.  Setting the minimum word count to 40 gave us a total vocabulary of 16,492 words with 300 features apiece. Individual word vectors can be accessed in the following way:




In [18]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")



2017-03-31 11:57:27,725 : INFO : loading Word2Vec object from 300features_40minwords_10context
2017-03-31 11:57:28,093 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2017-03-31 11:57:28,094 : INFO : setting ignored attribute syn0norm to None
2017-03-31 11:57:28,095 : INFO : setting ignored attribute cum_table to None
2017-03-31 11:57:28,096 : INFO : loaded 300features_40minwords_10context


In [19]:
type(model.syn1neg)



numpy.ndarray

In [20]:
model.syn1neg.shape

(16490, 300)

In [21]:
model["flower"]

array([  6.28269091e-02,   8.58594012e-03,   4.59386557e-02,
         1.51881836e-02,   5.19080684e-02,   1.25920132e-01,
        -5.71398512e-02,  -9.91477221e-02,  -1.72796529e-02,
         1.84542798e-02,   1.50005659e-02,  -7.92571809e-03,
         2.76399963e-03,   8.26712996e-02,  -5.04959095e-03,
        -3.66797261e-02,   5.57049364e-02,   2.05569919e-02,
         7.70074204e-02,   9.71768424e-02,  -1.09034948e-01,
         3.65568399e-02,   3.30877528e-02,  -1.51858047e-01,
         6.57621101e-02,  -6.93912581e-02,  -1.44826155e-02,
        -8.16756263e-02,   1.93370041e-02,   1.16986141e-01,
        -6.41599968e-02,   7.07110986e-02,   3.54566276e-02,
        -9.57761332e-03,   3.83535540e-03,  -6.46142066e-02,
        -2.50983052e-02,   6.05249368e-02,   6.22640178e-03,
         2.95700226e-02,  -1.24974832e-01,  -6.54374510e-02,
        -3.15378839e-03,   2.23963968e-02,   2.91006882e-02,
        -5.18014953e-02,  -1.63815036e-01,   1.64826829e-02,
        -2.34758686e-02,

## From Words To Paragraphs, Attempt 1: Vector Averaging

One challenge with the IMDB dataset is the variable-length reviews. We need to find a way to take individual word vectors and transform them into a feature set that is the same length for every review.

Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each review. One method we tried was to simply average the word vectors in a given review (for this purpose, we removed stop words, which would just add noise).

The following code averages the feature vectors, building on our code from Part 2. 




In [26]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    
    #print("in makefeature vec")
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
#            print(word)
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    #print("featureVec",featureVec)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # (13526, 300)
    
    # Loop through the reviews
    for review in reviews:
        ##### PROBLEM HERE. SEEMS TO ONLY GO THROUGH FIRST REVIEW 
        
#        print("in review")
        
        #
        # Print a status message every 1000th review
        if counter%1000 == 0:
            print ("Review %d of %d" % (counter, len(reviews)))
        # 
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
        #
        # Increment the counter
        counter = counter + 1
        #print("counter" , counter)
        
    return reviewFeatureVecs

In [25]:
# train["review"]
review_to_wordlist

<function __main__.review_to_wordlist>

Now, we can call these functions to create average vectors for each paragraph. The following operations will take a few minutes:



In [27]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [146]:
testDataVecs.shape

(25000, 300)

Next, use the average paragraph vectors to train a random forest. Note that, as in Part 1, we can only use the labeled training reviews to train the model. 

We found that this produced results much better than chance, but underperformed Bag of Words by a few percentage points.

Since the element-wise average of the vectors didn't produce spectacular results, perhaps we could do it in a more intelligent way? A standard way of weighting word vectors is to apply "tf-idf" weights, which measure how important a given word is within a given set of documents. One way to extract tf-idf weights in Python is by using scikit-learn's TfidfVectorizer, which has an interface similar to the CountVectorizer that we used in Part 1. However, when we tried weighting our word vectors in this way, we found no substantial improvement in performance.


        

In [28]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

# Your submission scored 0.82836.

Fitting a random forest to labeled training data...


In [49]:
testDataVecs.shape

(25000, 300)

In [48]:
trainDataVecs.shape

(25000, 300)

## From Words to Paragraphs, Attempt 2: Clustering 

Word2Vec creates clusters of semantically related words, so another possible approach is to exploit the similarity of words within a cluster. Grouping vectors in this way is known as "vector quantization." To accomplish this, we first need to find the centers of the word clusters, which we can do by using a clustering algorithm such as K-Means.

In K-Means, the one parameter we need to set is "K," or the number of clusters. How should we decide how many clusters to create? Trial and error suggested that small clusters, with an average of only 5 words or so per cluster, gave better results than large clusters with many words. Clustering code is given below. We use scikit-learn to perform our K-Means.

K-Means clustering with large K can be very slow; the following code took more than 40 minutes on my computer. Below, we set a timer around the K-Means function to see how long it takes.